In [1]:
import os
import pandas as pd 
from sklearn.model_selection import train_test_split

In [2]:
import torch

In [3]:
from transformers import AutoTokenizer

c:\Users\Dell\Desktop\NG_ChatRouter\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datasets import Dataset
import evaluate

In [5]:
MODEL_NAME = os.getenv("MODEL_NAME", "distilbert-base-uncased")
EPOCHS = int(os.getenv("EPOCHS", 3))
BATCH_SIZE = int(os.getenv("BATCH_SIZE", 16))
MAX_LENGTH = int(os.getenv("MAX_LENGTH", 128))
MODEL_DIR = os.getenv("MODEL_DIR", "saved_model")
metric = evaluate.load("f1")

In [6]:
def load_data(csv_path):
    df = df = pd.read_csv(csv_path, quotechar='"', escapechar='\\')
    assert "text" in df.columns and "intent" in df.columns and "department" in df.columns
    return df

In [7]:
def prepare_datasets(df):
    intents = sorted(df['intent'].unique().tolist())
    intent2id = {lab:i for i,lab in enumerate(intents)}
    id2intent = {v:k for k,v in intent2id.items()}
    print(intents)
    print(intent2id)
    print(id2intent)
    print(df.head())
    df["label"] = df["intent"].map(intent2id)
    # Note - Stratified split may fail if some classes have very few samples.
    # train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
    try:
        train_df, test_df = train_test_split(
            df, test_size=0.2, random_state=42, stratify=df["label"]
        )
    except ValueError as e:
        print("Stratified split failed:", e)
    print("Falling back to random split (no stratification).")
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    train_ds = Dataset.from_pandas(train_df[["text", "label"]])
    test_ds = Dataset.from_pandas(test_df[["text","label"]])
    return train_ds, test_ds, intent2id, id2intent
    

In [8]:
def tokenize_function(examples, tokenizer):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)

In [9]:
def compute_metrics(eval_pred):
    logits,labels = eval_pred
    preds = logits.argmax(-1)
    f1 = metric.compute(predictions=preds, references=labels, average="weighted")["f1"]
    acc = (preds == labels).mean()
    return {"accuracy": acc, "f1": f1}

In [10]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments


def train(csv_path, output_dir=MODEL_DIR):
    df = load_data(csv_path)
    train_ds, test_ds, intent2id, id2intent = prepare_datasets(df)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    train_ds = train_ds.map(lambda examples: tokenize_function(examples, tokenizer), batched=True)
    test_ds = test_ds.map(lambda examples: tokenize_function(examples, tokenizer), batched=True)
    train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    test_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(intent2id))

    training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir=f"{output_dir}/logs",
)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    trainer.train()
    os.makedirs(output_dir, exist_ok=True)
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    import json
    with open(os.path.join(output_dir, "intent2id.json"), "w") as f:
        json.dump(intent2id, f)
    with open(os.path.join(output_dir, "id2intent.json"), "w") as f:
        json.dump(id2intent, f)
    print("Training complete. Model saved to", output_dir)

In [11]:
if __name__ == "__main__":
    import argparse
    import sys
    parser = argparse.ArgumentParser()
    parser.add_argument("--csv", type=str, default="Data/example_data.csv")
    parser.add_argument("--output_dir", type=str, default="saved_model")

    # ignore unrecognized args (like --f)
    args, unknown = parser.parse_known_args()
    train(args.csv, args.output_dir)

['complaint', 'faq', 'general', 'profile_change', 'transaction_query']
{'complaint': 0, 'faq': 1, 'general': 2, 'profile_change': 3, 'transaction_query': 4}
{0: 'complaint', 1: 'faq', 2: 'general', 3: 'profile_change', 4: 'transaction_query'}
                                     text          intent        department
0       I want to change my email address  profile_change  account_services
1  My card was charged twice, need refund       complaint           billing
2              How do I reset my password             faq  account_services
3     I'd like to update my KYC documents  profile_change  account_services
4  I want to dispute a transaction of $50       complaint           billing
Stratified split failed: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.
Falling back to random split (no stratification).


c:\Users\Dell\Desktop\NG_ChatRouter\myvenv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 2/2 [00:00<00:00, 655.92 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\Dell\Desktop\NG_ChatRouter\myvenv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set a

{'train_runtime': 7.452, 'train_samples_per_second': 3.221, 'train_steps_per_second': 0.403, 'train_loss': 1.574869155883789, 'epoch': 3.0}
Training complete. Model saved to saved_model


In [12]:
import json
from loguru import logger

In [13]:
ROUTING_TABLE  = {
    "profile_change": "account_services",
    "complaint":"billing",
    "faq":"customer_support",
    "transation_query":"transactions",
    "general":"customer_support",
    "loans":"loans_officer"        
}

def rule_based_override(metadata):
    if(metadata.get("is_vip")):
        return "priority_support"
    if(metadata.get("customer_tier") == "gold"):
        return "priority_support"
    return None

def map_intent_to_department(intent_label):
    return ROUTING_TABLE.get(intent_label, "customer_support")

def decide_route(predicted_intent, confidence, metadata):
    override = rule_based_override(metadata)
    if override:
        logger.info(f"Rule override to {override} based on metadata {metadata}")
        return override, "rule_override"
    dept = map_intent_to_department(predicted_intent)
    return dept, "intent_mapping"

In [14]:
# test_inference.py
import os
import json
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# === Config ===
MODEL_DIR = "saved_model"
CONFIDENCE_THRESHOLD = 0.7

# === Load model, tokenizer, and label maps ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()

with open(os.path.join(MODEL_DIR, "id2intent.json"), "r") as f:
    id2intent = json.load(f)

# === Helper: predict intent & confidence ===
def predict_intent(text):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1).squeeze().cpu().numpy()
    top_idx = int(probs.argmax())
    confidence = float(probs[top_idx])
    intent_label = id2intent.get(str(top_idx), id2intent.get(top_idx, "unknown"))
    return intent_label, confidence, probs

# === Test examples ===
examples = [
    {"text": "I want to change my email address", "metadata": {}},
    {"text": "My card was charged twice", "metadata": {"is_vip": False}},
    {"text": "How to apply for a loan?", "metadata": {"customer_tier": "gold"}},
    {"text": "Please reset my password", "metadata": {}},
]

print(f"\n{'='*20} Running Routing Tests {'='*20}\n")
for ex in examples:
    intent, confidence, probs = predict_intent(ex["text"])
    clarification_needed = confidence < CONFIDENCE_THRESHOLD
    if clarification_needed:
        dept = None
        decision_reason = "low_confidence_clarification"
    else:
        dept, decision_reason = decide_route(intent, confidence, ex["metadata"])

    print(f"Text: {ex['text']}")
    print(f" → Predicted Intent: {intent}")
    print(f" → Confidence: {confidence:.3f}")
    print(f" → Clarification Needed: {clarification_needed}")
    print(f" → Department: {dept}")
    print(f" → Reason: {decision_reason}")
    print("-" * 60)



==================== Running Routing Tests ====================

Text: I want to change my email address
 → Predicted Intent: faq
 → Confidence: 0.224
 → Clarification Needed: True
 → Department: None
 → Reason: low_confidence_clarification
------------------------------------------------------------
Text: My card was charged twice
 → Predicted Intent: faq
 → Confidence: 0.226
 → Clarification Needed: True
 → Department: None
 → Reason: low_confidence_clarification
------------------------------------------------------------
Text: How to apply for a loan?
 → Predicted Intent: faq
 → Confidence: 0.226
 → Clarification Needed: True
 → Department: None
 → Reason: low_confidence_clarification
------------------------------------------------------------
Text: Please reset my password
 → Predicted Intent: faq
 → Confidence: 0.222
 → Clarification Needed: True
 → Department: None
 → Reason: low_confidence_clarification
------------------------------------------------------------
